### Import the library 

In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sin, cos, sqrt, atan2, radians
# import pymc3 as pm

### Import the data

In [63]:
df = pd.read_csv("nyc-taxi-trip-duration/train.csv")
display(df.shape)
df.head()

(1458644, 11)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [64]:
wall_street_location = [40.70729,-74.01095]
# sample_location = [[40.73,-74.05],[40.665,-73.795],[40.77,-73.995],[40.74,-73.99],[40.695,-73.95],[40.767,-73.875]]
all_location = [[40.73,-74.05],[40.665,-73.795],[40.77,-73.995],[40.74,-73.99],[40.695,-73.95],[40.767,-73.875],[40.70729,-74.01095]]
all_location_label = ["1","2","3","4","5","6","HQ"]

dict_location = dict(zip(all_location_label, all_location))


In [65]:
def get_distance_between_points(lat1,lon1,lat2,lon2):
    '''
    This function is used to get the distance between lat1, lon1 and lat2, lon2
    '''
    # approximate radius of earth in km
    R = 6378.0
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [66]:
# calculate the distance between the location of wall street to others point in the dataset.
df['dist_pickup_to_wall_street'] = np.vectorize(get_distance_between_points)(wall_street_location[0],wall_street_location[1],df['pickup_latitude'], df['pickup_longitude'])


df['dist_pickup_to_dropoff'] = np.vectorize(get_distance_between_points)(df['dropoff_latitude'],df['dropoff_longitude'],df['pickup_latitude'], df['pickup_longitude'])


In order to extract the most relevant information from the
dataset for our purpose, we create a filtered dataset, in
which either the pick-up or drop-off location lies within
500 m from Wall Street (indicated by the red oval in Fig.
3(b)). In addition, we filter trips with unrealistically long
durations (>20 000 s).

In [67]:
df_wall = df[( df['dist_pickup_to_wall_street'] <= 0.5)& (df['trip_duration'] < 20000)]
df_wall.shape



(20535, 13)

### Choosing the prediction model

simply use 5 different prediction algorithm as the model, and compare its performance.

From this data, we will only use its location (dropoff_longitude and dropoff_latitude) and distance from the HQ to predict the trip duration.

In [68]:
 
temp_df = df_wall[['dropoff_latitude','dropoff_longitude','dist_pickup_to_dropoff','trip_duration']].copy()

# Declare the model
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import BaggingRegressor
from sklearn import model_selection
from sklearn.model_selection import train_test_split
experiments = {"Algorithm":["Ordinary Least Squares", "Regression Tree", 
                            "Big Random Forest", "Random Forest", 
                            "Bagging"], 
               "Objects" : [lambda : LinearRegression(), 
                            lambda : DecisionTreeRegressor(), 
                            lambda : RandomForestRegressor(n_estimators=100), 
                            lambda : RandomForestRegressor(), 
                            lambda : BaggingRegressor()], 
               "Predictions":[[] for _ in range(5)]}
actuals = []

In [69]:
# run the 5 differenet algorithm and compare the result

for _ in range (4):
    train_X, test_X, train_y, test_y = (
        train_test_split(temp_df.drop("trip_duration", axis=1), 
                         temp_df["trip_duration"], test_size=0.33))
    for i, obj_factory in enumerate(experiments["Objects"]):
        obj = obj_factory()
        obj.fit(y=train_y,X=train_X)
        experiments["Predictions"][i] += list(obj.predict(test_X))
    actuals += list(test_y)
actuals = pd.Series(actuals)
experiments["Predictions"] = list(map(pd.Series, experiments["Predictions"]))

# Get the performance of every model
experiments["Results"] = []
for o in experiments["Objects"]:
    experiments["Results"].append(
        model_selection.cross_val_score(o(), y=temp_df['trip_duration'], 
                                        X=temp_df.drop("trip_duration", axis=1),
                                        cv=5).mean())

pd.DataFrame(experiments).drop(["Objects", "Predictions"], 
                            axis=1).set_index("Algorithm")

/Users/lilianaester/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lilianaester/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lilianaester/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lilianaester/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lilianaes

,Results
Algorithm,
Ordinary Least Squares,0.514720
Regression Tree,0.166908
Big Random Forest,0.529410
Random Forest,0.500419
Bagging,0.496262


Big random forest performance very well compare to the other. So, in this case, we will use big random forest as the algorithm for predicting trip duration. To easily re-produce the model, it is important to create the model in a function.

In [70]:
def create_model(temp_df, wall_street_location):
    '''
    parameter : 
        - temp_df = the training dataframe
        - wall_street_location = the headquarter
    '''
    train_X, test_X, train_y, test_y = ( train_test_split(temp_df.drop("trip_duration", axis=1), 
                             temp_df["trip_duration"], test_size=0.33))

    obj = RandomForestRegressor(n_estimators=100)
    obj.fit(y=train_y,X=train_X)
#     obj.predict(test_X)
    
#     print("Cross val score : ",model_selection.cross_val_score(obj, y=temp_df['trip_duration'], 
#                                             X=temp_df.drop("trip_duration", axis=1),
#                                             cv=5).mean())
#     obj.predict(temp_df.drop("trip_duration", axis=1))

    ## CREATE THE SAMPLE FOR THE MODEL
    sample_lat = []
    sample_lng = []
    sample_distance = []

    for i in dict_location:
        if int(i) > 5:
            break
        sample_lat.append(dict_location[i][0])
        sample_lng.append(dict_location[i][1])
        sample_distance.append(get_distance_between_points(wall_street_location[0],wall_street_location[1],dict_location[i][0],dict_location[i][1]))
        
    
    d = {'dropoff_latitude' : sample_lat,
     'dropoff_longitude' : sample_lng,
    'dist_pickup_to_dropoff': sample_distance}
    temp_pred_df = pd.DataFrame(d)  
    
    temp_pred_df['predicted_duration'] = obj.predict(temp_pred_df)
    return temp_pred_df

In [71]:
def get_location_names(lat,lng):
    for k,v in dict_location.items():
        if  (v[0] == lat) & (v[1] == lng) :
            return k
        

In [72]:
df1 = create_model(temp_df,dict_location["1"])
df1

,dropoff_latitude,dropoff_longitude,dist_pickup_to_dropoff,predicted_duration
0,40.730,-74.050,0.000000,1309.503333
1,40.665,-73.795,22.704878,2566.200000
2,40.770,-73.995,6.429525,1352.170000
3,40.740,-73.990,5.181914,1590.750000
4,40.695,-73.950,9.293823,1833.190000


In [73]:
prediction_list = []
for i in dict_location:
    if int(i) >5:
        break
    tdf = create_model(temp_df,dict_location[str(i)])
    prediction_list.append(list(tdf['predicted_duration'].apply(lambda x : int(x))))
    print(list(tdf['predicted_duration'].apply(lambda x : int(x))))

[924, 2955, 1546, 1479, 1860]
[2323, 1249, 2602, 2090, 2188]
[2190, 2206, 484, 839, 1983]
[2489, 3053, 911, 1093, 1239]
[1625, 2989, 1974, 1531, 1302]


In [74]:
distance_list = []
for i in dict_location:
    if int(i) > 5:
        break
    tdf = create_model(temp_df,dict_location[str(i)])
    distance_list.append(list(tdf['dist_pickup_to_dropoff'].round(3)))
    print(list(tdf['dist_pickup_to_dropoff'].round(3)))
    

[0.0, 22.705, 6.43, 5.182, 9.294]
[22.705, 0.0, 20.527, 18.453, 13.504]
[6.43, 20.527, 0.0, 3.366, 9.171]
[5.182, 18.453, 3.366, 0.0, 6.04]
[9.294, 13.504, 9.171, 6.04, 0.0]


In [75]:
for i in range(len(prediction_list)):
    prediction_list[i][i] = 0
prediction_list


[[0, 2955, 1546, 1479, 1860],
 [2323, 0, 2602, 2090, 2188],
 [2190, 2206, 0, 839, 1983],
 [2489, 3053, 911, 0, 1239],
 [1625, 2989, 1974, 1531, 0]]

In [76]:
%load_ext iminizinc

The iminizinc extension is already loaded. To reload it, use:
  %reload_ext iminizinc


In [77]:
dist = prediction_list
NumLocs= 5;

In [78]:
%%mzn_model taxi_model
%
% Vehicle Routing Problem
%

% Location data

% Each customer is represented as a pair of x and y coordinates.
% The final location is that of the depot.

% x coordinates of location
% array[int] of float: locX;
% % y coordinates of location
% array[int] of float: locY;


% Locations
int: NumLocs;
set of int: Locations = 1..NumLocs;

array [Locations, Locations] of int: dist;


% Survey
int: N = NumLocs - 1;

% Vehicles
int: NumRoutes = 4*N; % Worst Case 
set of int: Vehicles = 1..NumRoutes;  % One Vehicle for each Route

% Survey Time
% We assume that the amount of time it takes to do a survey is constant, e.g. 70000 seconds
int: surveyTime = 1800;


% % Euclidian distance between locations, computed from
% % x/y coordinates
% array [Locations, Locations] of int: dist = 
%     array2d (
%         Locations, Locations,
%         [ round (1000.0 * sqrt( (locX[a] - locX[b]) * (locX[a] - locX[b]) +
%                                 (locY[a] - locY[b]) * (locY[a] - locY[b]))
%           )
%         | a, b in Locations
%         ]
%     );




% =====================================
% Model
% =====================================

include "globals.mzn";

% Define sets of nodes

% Survey locations have 2 states, a start and an end state
set of int: ArriveSurvey = 1..N;
set of int: LeaveSurvey = N+1..2*N;
% 1..2*N represent survey nodes, anything greater is a Depot Node  
% At the worst case, A taxi will visit all survey nodes
set of int: TaxiStart1 = 2*N+1..4*N;  % All Starting Nodes for Taxi 1
set of int: TaxiStart2  = 4*N+1..6*N; % All Starting Nodes for Taxi 2
set of int: TaxiEnd1 = 6*N+1..8*N; % All Ending Nodes for Taxi 1
set of int: TaxiEnd2 = 8*N+1..10*N; % All Ending Nodes for Taxi 2

% All Nodes
set of int: Visits = 1..max(TaxiEnd2);
% All Survey Nodes plus Taxi Starting Nodes
set of int: SurveyPlusStarts = 1..max(TaxiStart2);


% Array of Distances between 2 points i and j
% Extended array of distances, including the depot start/end nodes.
array[Visits,Visits] of int: visit_dist = array2d(Visits,Visits,
  [if i> 2 * N /\ j > 2 * N then 0  % Distance between Depots
   elseif i > 2 * N then dist[NumLocs,(j mod N) + 1] % Distance between Depots and Survey (i > Survey)
   elseif j> 2 * N then dist[(i mod N)+1,NumLocs] % Distance between Survey and Depot (j > Survey)
   elseif i + N = j then 0
   elseif j + N = i then 0               % Distance between the same depots is 0
   else dist[(i mod N)+1 ,(j mod N)+1] endif | i,j in Visits]); % Euclidean Distance for the rest

% =====================================
% Decision variables
% =====================================

% Successor representation
array[Visits] of var Visits: successor;

% Time (i-th element is the time when the taxi visits the i-th node)
array[Visits] of var 0..infinity: time;
% Time (i-th element is the time when the i-th survey starts)
array[ArriveSurvey] of var 0..infinity: surveyStart;

% Mapping from visits to route (vehicle) number
array[Visits] of var Vehicles: routeOf;

% Objective function:
% Maximum time taken to complete all surveys
var int: objective = max(time);

% =====================================
% Problem Constraints
% =====================================

% Route & Successor Constraints
% =====================================

% Assign each vehicle depot start node to its route
constraint forall (k in 1..4*N) ( routeOf[2*N + k] = k);

% Assign each vehicle depot end node to its route
constraint forall (k in 1..4*N) ( routeOf[6*N + k] = k);

% The successors form a circuit (grand tour representation)
constraint circuit (successor) ::domain;

% successor of end is start of next, wraps around to first depot node
constraint
  forall (k in 1..4*N)
    ( successor[2 * N + 4 * N + k] = ( 2*N + 1 + (k mod NumRoutes) ) );

% Successor of starting survey node can only be a depot node or an ending survey node
constraint forall(k in ArriveSurvey)(successor[k] > N);
% Sucessor of ending survey node can only be a depot node or a starting survey node
constraint forall(k in LeaveSurvey)(successor[k] <= N \/ successor[k] >= 2*N + 1);
    
% Link up the routeOf and successor variables:
% Each visit is on the same route as its successor.
% This is true for all survey and start visits, but not for the end visits.
constraint forall (i in SurveyPlusStarts) ( routeOf[i] = routeOf[successor[i]] );


% =====================================
% Survey Constraints
% =====================================

% Survey i should start after or at the time the survey team reaches survey site i
constraint forall(i in ArriveSurvey)(surveyStart[i] >= time[i]);
% The time the taxi leaves from survey site i should be after 1800 seconds from the time the survey starts
constraint forall(i in ArriveSurvey)(surveyStart[i] + surveyTime <= time[i + N]);

% This constraint makes sure that at any given time, only 2 surveys maybe be done in parallel
% It takes in an array of start times, the amount of time it takes to finish the survey, how many survey teams are used for that survey, and the maximum number of survey teams available
constraint cumulative(surveyStart, [surveyTime | i in 1..N], [1 | i in 1..N], 2);

% =====================================
% Time Constraints
% =====================================

% Accumulate Time
constraint forall(i in Visits where i != 4 * N - 1 /\ i != 6 * N - 1)(time[successor[i]] >= time[i] + visit_dist[i, successor[i]]);

% Starting Taxi times are 0, at 2n + 1 and 4n + 1
constraint time[2 * N + 1] = 0;
constraint time[4 * N + 1] = 0;

% Ending Depot Time should be the same as the next Starting Depot Time, for each Taxi
constraint forall(i in 1..2, j in 1..2*N-1)(time[6*N + j + (i-1)*N] = time[2*N + j + 1 + (i-1)*N]);
  

% =====================================
% Symmetry breaking
% =====================================

% If the successor of the i-th starting taxi node is its ending taxi node, then the successor of 
% the (i+1)-th starting taxi node will also be its corresponding ending taxi node
constraint forall(i in 1..2*N-1) (successor[2*N+i] = 6*N + i -> successor[2*N+i+1] = 6*N + i + 1);

% =====================================
% Solve and output
% =====================================

% We want to minimize the time taken to complete all surveys
solve
  :: int_search ( successor, dom_w_deg, indomain_min, complete)
  minimize (objective);

output
[ "duration = ", show(objective), "\n" ] ++
[ "successor = ", show(successor), ";\n" ] ++
[ "routeOf = ", show(routeOf), ";\n"] 
++ [ "time = ", show(time), ";\n"]
++ [ "surveyStart = ", show(surveyStart), ";\n"];

In [ ]:
output = %minizinc taxi_model --solver chuffed
print(output)

In [153]:
# import libraries
import folium
import pandas as pd
 
lat_list = []
lon_list= []
for i in dict_location:
    lat_list.append(dict_location[i][0])
    lon_list.append(dict_location[i][1])
    
# Make a data frame with dots to show on the map
data = pd.DataFrame({
'lat': lat_list,
'lon': lon_list,
'name':['HQ','1','2','3','4','5','6']
})
data
 
# Make an empty map
# m = folium.Map(location=[20, 0], tiles="Mapbox Bright", zoom_start=2)
m = folium.Map(location=wall_street_location, tiles="Mapbox Control Room")

 
# I can add marker one by one on the map
for i in range(0,len(data)):
    folium.Marker([data.iloc[i]['lat'], data.iloc[i]['lon']], popup=data.iloc[i]['name']).add_to(m)

m